**извлечение признаков из текста на естественном языке**

очистка текста и токенизация

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import re
# import numpy as np
# import numpy.random as rng
import pandas as pd
from tqdm import tqdm

# np.set_printoptions(precision=2) # вывод на печать чисел до 2 знака
pd.options.display.max_colwidth = 200 

tqdm.pandas()

/usr/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## тексты

In [2]:
# загружаем тексты
data = pd.read_pickle('../data/news.pkl.gz')
print('записей:',len(data))
data.sample(5)

записей: 3196


,text,tag
3093,ИА Dv-News (dv-news.com) Тувинцы-тоджинцы собрались в общественную\nорганизацию Управление министерства юстиции Республики Тува зарегистрировало\nв качестве общественной организации ассоциацию кор...,politics
1207,"""Дорога на Драгобрат"": как мы два раза пытались сломать Hyundai Creta\n\n9 декабря 2016 в 7:53\n\nДмитрий Новицкий, AUTO.TUT.BY\n\nКак сломать «паркетник», небольшой кроссовер? Отправить его по ма...",auto
1556,"11:01 сегодня 83 0\n\nФото: ura.ru\n\nПо словам главы ведомства Александра Бортникова, за указанный период были нейтрализованы 129 боевиков, в том числе 22 главаря бандподполья.\n\nЗадержаны 898 б...",politics
1732,"В ходе торгов на Интерконтинентальной бирже стоимость февральских фьючерсов на североморскую нефтяную смесь марки Brent по состоянию на 7:14 по московскому времени поднялась на 2,477% — до $55,71 ...",economics
1311,"Минчанина, тайно ночевавшего в батутной арене, уличили в обмане (обновлено)\n\n6 декабря 2016 в 12:12\n\n42.TUT.BY\n\nРолик, где два молодых парня ночуют в одном из батутных центров Минска, оказал...",tech


In [3]:
# длина строк
pd.DataFrame(data['text'].str.len()).describe([.1,.25,.5,.75,.95]).astype(int).T

,count,mean,std,min,10%,25%,50%,75%,95%,max
text,3196,1720,2177,25,370,675,1070,1813,5761,30710


In [4]:
# количество категорий
data['tag'].drop_duplicates().count()

13

In [5]:
# собираем словарь из текстов
def get_vocabulary(ds):
    vcb = [ set(s) for s in ds.tolist() ]
    return sorted(set.union(*vcb))

## очистка и токенизация 

In [6]:
# применяет список замен pat к строке s
def replace_patterns(s,pat):
    if len(pat)<1: return s
    return  replace_patterns( re.sub(pat[0][0],pat[0][1],s), pat[1:] )

# нормализация текста
def string_normalizer(s):
    pat = [
       [r'ё','е'] # замена ё для унификации
       ,[r'</?[a-z]+>',' '] # удаляем xml
       ,[r'[^a-zа-я\- ]+',' '] # оставляем только буквы, пробел и -
       ,[r' -\w+',' '] # удаляем '-й','-тый' и т.п.
       ,[r'\w+- ',' ']
       ,[r' +',' '] # удаляем повторы пробелов
    ]
    return replace_patterns(s.lower(),pat).strip()

# разрезаем стоки на слова
def tokenize(s): 
    return [ w for w in s.split(' ') if (len(w)>1) ]

In [7]:
data['ctext'] = data['text'].progress_apply(string_normalizer).progress_apply( tokenize )

100%|██████████| 3196/3196 [00:00<00:00, 36052.96it/s]


In [8]:
vcb =  get_vocabulary( data['ctext'] )
print('словарь %i слов'%(len(vcb)))
# pd.DataFrame( vcb ).to_csv('voc0.txt',index=False,header=False)

словарь 83094 слов


In [9]:
data.sample(3)

,text,tag,ctext
481,Исследование: США и Западная Европа продают четыре пятых всего оружия в мире\n\n5 декабря 2016 в 8:19\n\nDeutsche Welle\n\nОбъем продаж оружия и услуг сотни крупнейших военных концернов мира в 201...,politics,"[исследование, сша, западная, европа, продают, четыре, пятых, всего, оружия, мире, декабря, deutsche, welle, объем, продаж, оружия, услуг, сотни, крупнейших, военных, концернов, мира, году, состав..."
3024,"140 млн. рублей направит <object>МРСК Сибири</object> в 2010 году на приобретение костюмов, защищающих персонал от влияния электрической дуги.\n\nПроизводственной безопасности и охране труда в <ob...",social,"[млн, рублей, направит, мрск, сибири, году, на, приобретение, костюмов, защищающих, персонал, от, влияния, электрической, дуги, производственной, безопасности, охране, труда, мрск, сибири, уделяет..."
2945,"Так Audi Q3 нового поколения предстанет как с обычным бензиновым, либо дизельным мотором. А также для китайского рынка кроссовер выйдет с гибридным двигателем.Автокомпания Audi приняла решение вне...",auto,"[так, audi, нового, поколения, предстанет, как, обычным, бензиновым, либо, дизельным, мотором, также, для, китайского, рынка, кроссовер, выйдет, гибридным, двигателем, автокомпания, audi, приняла,..."


## стеминг

In [10]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords
# from nltk import download as nltk_download
# nltk_download('stopwords')

stopwords = set(nltk_stopwords.words('russian'))
stemmer = SnowballStemmer('russian')

# выкидываем stopwords, выполняем стеминг
def stem(s): 
    return [ stemmer.stem(w) for w in s if w not in stopwords ]

In [11]:
data['ctext'] = data['ctext'].progress_apply( stem )

100%|██████████| 3196/3196 [00:15<00:00, 208.14it/s]


In [12]:
vcb =  get_vocabulary( data['ctext'] )
print('словарь %i слов'%(len(vcb)))
# pd.DataFrame( vcb ).to_csv('voc1.txt',index=False,header=False)

словарь 36381 слов


In [13]:
data.sample(3)

,text,tag,ctext
94,"Использование шифрованного Flash диска под LinuxВ статье "" Encrypt devices\nusing dm-crypt and LUKS "" рассказывается о создании шифрованного раздела\nна мобильном устройстве, например USB Flash.",tech,"[использован, шифрова, flash, диск, линуxв, стат, encrypt, devices, using, dm-crypt, and, luks, рассказыва, создан, шифрова, раздел, мобильн, устройств, например, usb, flash]"
417,Украина получила официальное уведомление о статусе Януковича в РФ\n\n9 декабря 2016 в 16:14\n\nРБК-Украина\n\nГенеральная прокуратура Украины получила официальное уведомление о статусе экс-президе...,politics,"[украин, получ, официальн, уведомлен, статус, янукович, рф, декабр, рбк-украин, генеральн, прокуратур, украин, получ, официальн, уведомлен, статус, экс-президент, виктор, янукович, российск, федер..."
1435,"Бывший губернатор Сахалинской области Александр Хорошавин, его жена Ирина и сын Илья подали в Конституционный суд жалобу, оспаривающую конфискацию имущества до окончания расследования. По данным г...",politics,"[бывш, губернатор, сахалинск, област, александр, хорошавин, жен, ирин, сын, ил, пода, конституцион, суд, жалоб, оспарива, конфискац, имуществ, окончан, расследован, дан, газет, коммерсант, жалоб, ..."


## лемматизация

In [14]:
# лемматизация и очистка с помощью пакета морфологического анализа

from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

# NOUN (существительное), VERB (глагол), ADJF (прилагательное)
def word_normalizer(w, pos_types=('NOUN','VERB','ADJF')):
    if not morph.word_is_known(w): return ''
    p = morph.parse(w)[0] 
    return p.normal_form if (p.tag.POS in pos_types) else ''

def text_normalizer_tokenize(s):
    return [ word_normalizer(w) for w in s.split(' ') if len(w)>1 ]

In [16]:
data['ctext'] = data['text']\
                   .progress_apply(string_normalizer)\
                   .progress_apply( text_normalizer_tokenize )

100%|██████████| 3196/3196 [01:12<00:00, 43.88it/s]


In [17]:
vcb =  get_vocabulary( data['ctext'] )
print('словарь %i слов'%(len(vcb)))
# pd.DataFrame( vcb ).to_csv('voc2.txt',index=False,header=False)

словарь 22123 слов


In [18]:
data.sample(3)

,text,tag,ctext
1857,"Президент России Владимир Путин в интервью японскому телеканалу Nippon TV заявил, что РФ хочет полной нормализации отношений с Японией и считает отсутствие мирного договора между двумя странами ан...",politics,"[президент, россия, владимир, путин, интервью, японский, телеканал, , , заявить, , рф, хотеть, полный, нормализация, отношение, япония, считать, отсутствие, мирный, договор, , , страна, анахронизм..."
279,"""Уралсвязьинформ"" готов к переходу на новые коды... Министерства\nинформационных технологий и связи РФ ""Об изменении закрепления кодов\nгеографически определяемых зон нумерации и кодов географичес...",tech,"[уралсвязьинформ, , переход, , новое, код, министерство, информационный, технология, связь, рф, , изменение, закрепление, код, , , зона, нумерация, код, , неопределяемый, зона, , междугородный, те..."
703,"Песни Мулявина, вышиванки и победитель из провинции. В Dozari прошел ""Студент года"" от БРСМ\n\n9 декабря 2016 в 12:52\n\nМайя Кохно / Фото: Евгений Ерчак / TUT.BY\n\nВ четверг, 8 декабря, в столич...",politics,"[песнь, , , победитель, , провинция, , пройти, студент, год, , , декабрь, майя, , фото, евгений, , , , четверг, декабрь, столичный, клуб, , пройти, финал, конкурс, студент, год, , , звание, соревн..."
